# <font color='#FFBD33'>**Assignment 4 - Auto Correction**</font> 

This is <font color='cyan'>Assignment 4</font> for the LING360 - Computational Methods in Lingustics course and it is worth a total of  <font color='cyan'>**5 points**</font>.
The assignment covers edit distance and its utilization. 

The topics include:
1. Regular Expressions
1. Edit Distance


There's a total of  <font color='cyan'>**2 main tasks**</font> and <font color='cyan'>**5 subs tasks**</font>. For each task, please write your code between the following lines:

```
## YOUR CODE STARTS



## YOUR CODE ENDS
```

Before working on the assignment, please copy this notebook to your own drive. You can use ```Save a copy in Drive``` under the ```File``` menu on top left.

Please, run every cell in your code to make sure that it works properly before submitting it. 

Once you are ready to submit, download two versions of your code:

*   Download .ipynb
*   Download .py

These are both available under the ```File``` menu on top left. 

Then, compress your files (zip, rar, or whatever) and upload the compressed file to Moodle.

If you have any questions, please contact with karahan.sahin@boun.edu.tr


In [1]:
# FIRST, RUN THIS LINE (IMPORTING CORPUS FILE FROM THE SOURCE)
! wget https://github.com/maidis/turkish-parallel-corpora/blob/master/kde4/kde4_tr_trunk_20181124.tr

--2023-04-13 07:10:41--  https://github.com/maidis/turkish-parallel-corpora/blob/master/kde4/kde4_tr_trunk_20181124.tr
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘kde4_tr_trunk_20181124.tr’

kde4_tr_trunk_20181     [ <=>                ]   6.24M  --.-KB/s    in 0.1s    

2023-04-13 07:10:42 (52.9 MB/s) - ‘kde4_tr_trunk_20181124.tr’ saved [6544953]



## <font color='#FFBD33'>**Q1:** Get and Clean Data</font> `2 points`

In computational linguistics, you might need to deal with different data sources, and most of them will be dirty and contain HTML tags, emojis, and other unnecessary content. Therefore, you need deal with them before starting to work on your data.

### <font color='#FFBD33'>**Q1.1:** Read and Clean Corpus</font> `0.5 Points`

Just read the data and clean html tags in the data. Also lower all the characters so that we don't have to deal with sentence beginning with different versions of the same words.

<font color='#FFBD33'>**Instructions:**</font>

1. Import and read file `kde4_tr_trunk_20181124.tr` using the `open()` function and assign it to a variable called `corpus`, as we have seen previously.
1. Then clean the HTML tags within the text via `re.sub()` function.
1. Lower all characters in the corpus.


In [19]:
## YOUR CODE STARTS
import re
corpus = open("kde4_tr_trunk_20181124.tr", "r", encoding="utf-8" ).read()

corpus = re.sub(r"<.*?>+"," ", corpus)
corpus = corpus.lower()
## YOUR CODE ENDS

### <font color='#FFBD33'>**Q1.2:** Tokenize Corpus</font> `0.5 Points`

Tokenize all the words after cleaning corpus the from the html tags. While tokenizing the data, remove tokens which only consist of numbers, punctuations, or single letter characters. 

<font color='#FFBD33'>**Instructions:**</font>

1. Either tokenize via by custom function as we did in Assignment 3 or use `nltk.wordpunkt_tokenize()` function to tokenize your corpus.
1. Assign tokenized corpus into `tokens` variable
1. Define a variable named `cleaned_tokens` which is an empty list
1. Iterate over `tokens` list and only append those that are not
  1. Only consist of numbers `token.isnum() == True`
  1. Only consist of punctuations `token.isalpha() != True`
  1. Only consist of single character `len(token) =< 1`


In [21]:
## YOUR CODE STARTS
from nltk.tokenize import wordpunct_tokenize
tokens = set(wordpunct_tokenize(corpus))

cleaned_tokens = []

for token in tokens:
  if not token.isnumeric():
    cleaned_tokens.append(token)
  elif token.isalpha():
    cleaned_tokens.append(token)
  elif len(token) > 1:
      cleaned_tokens.append(token)
## YOUR CODE ENDS

### <font color='#FFBD33'>**Q1.3:** Get Token Counts</font> `1 Points`

Get the token counts and sort them in a descending order (we did this in Lab 5). The token counts will be used to provide the best suggestion by identifying the most frequent words in the corpus. When making a suggestion using edit distance, we want to pick the most frequent word.

<font color='#FFBD33'>**Instructions:**</font>
1. Create a new `Counter()` object with `cleaned_tokens` as its argument and assign it to a variable named `token_count`.
2. Then sort the `token_count` by its counts in the corpus using the `sorted` function in descending order as we have seen in Lab 5.
3. Finally turn it into a list again and assign it to a variable named `sorted_keywords`.


In [22]:
from collections import Counter

## YOUR CODE STARTS
token_count = Counter(cleaned_tokens)

sorted_keywords = list(sorted(token_count.items(), key=lambda x: x[1], reverse=True))
## YOUR CODE ENDS

## <font color='#FFBD33'>**Q2:** Auto Correction</font> `3 points`

Now that we have the data, you can implement an auto correction module. But you don't just recommend suggestions with minimum edit distance but you will be improving them according the statistical information in your corpus. Also you will be saving the previously selected corrections. 
You will be using pre-defined edit distance library so that you don't have to implement it from scratch. 


In [23]:
#install the editdistance library
!pip install editdistance --quiet

In [24]:
## Example usage 
# You don't have to re-invent everything on your own, just find the library for it :) 
import editdistance
editdistance.distance("gelebildi", "gelemedi")

3

### <font color='#FFBD33'>**Q2.2:** Recommend with Edit Distance</font> `1.5 Points`

Write your suggestion function using the algorithm below.

<font color='#FFBD33'>**Instructions:**</font>
1. First, check whether you have the word in your corpus. If it is, return a list containing only your target `word`.
1. If it is not available in your corpus, check the edit distance between your target word and each word in the corpus using a `for` loop.
1. After saving the edit distance between each word in the corpus and your target word, find the minimum edit distance and assign it to a variable named `min_dist`.
1. Define an empty list called `suggestions`.
1. Then iterate over `sorted_keywords` and add suggested words (i.e. words with the minimum edit distance from your target word) to the `suggestions` list until the list has a length of `n_suggestions`, which is a parameter determining how many words will be suggested by your function.
1. Finally return the `suggestions` list.

In [50]:
def getSuggestionsWithEditDistance(word, n_suggestions):
    """
    Suggest number of candidates which for a given possible incorrect word.
    """
    ## YOUR CODE STARTS
    candidates = []
    if word in sorted_keywords:
      candidates.append(word[0])
      return candidates
    else:
      distance_list = []
      for candidate in sorted_keywords:
        distance = editdistance.distance(candidate[0], word)
        distance_list.append(distance)
      
      min_dist = min(distance_list)

      suggestions = []
      for candidate in sorted_keywords:
        if min_dist == editdistance.distance(candidate[0], word):
          suggestions.append(candidate[0])
          if len(suggestions) == n_suggestions:
            break
        
      return suggestions
    ## YOUR CODE ENDS

In [51]:
getSuggestionsWithEditDistance("gelmesm", 3)

['gelmesi']

### <font color='#FFBD33'>**Q2.3:** Final Product</font> `1.5 Points`

Final product can be utilized in everyday life, so you will be writing a mini application on top of it. 

<font color='#FFBD33'>**Instructions:**</font>
1. Take input from the user via using `input()` function and assign it to a variable named `candidate`.
1. Then run the `getSuggestionsWithEditDistance()` where `n_suggestion=5` for input word, and .
1. Print each option with its correct index via using the `enumerate()` function as "`index`. `suggestion`".
1. Ask the index from the user and save the corresponding incorrect-correct match to `MEMORY` dictionary.
1. Go back to the top, first try to find the `word` in `MEMORY`, so that you don't need to calculate over and over again.
1. If it is in `MEMORY`, print "The correct word is `your_word`".

<font color='#FFBD33'>**Note:**</font>
1. `input()` function returns only strings, so if you are giving numerical input, please translate to integer via `int()`.
2. Do not print suggestions if you have an answer in the `MEMORY`.
3. Example Usage of `enumerate()`
  ```python
  my_list = ["a", "b", "c"]
  for i in enumerate(my_list):
    print(i)

  # Output:
  # (0, "a")
  # (1, "b")
  # (2, "c")
  ```

###  <font color='#FFBD33'>Example Outputs</font>
1. First Example

```bash
Your word is: gelmesm
Selected corrected version:
0. gelemedi
1. gelmedi
2. gelmesi
3. gelmezdi

Please give your response: 3
Your answer is saved as "gelmesm -> gelmezdi"
```

2. After first example

```bash
Your word is: gelmesm
The correct word is "gelmezdi"
```

In [52]:
MEMORY = {}

In [53]:
def AutoCorrect(word):
    """
    AutoCorrect function translates incorrect words into correct
    -i: 
      word:string: incorrect word candidate
    :returns: None
    """
    ## YOUR CODE STARTS
    candidate = input('Your word is: ')
    if candidate in MEMORY:
      print('The correct word is  "{}"'.format(MEMORY[candidate]))
    else:
      suggested_words = getSuggestionsWithEditDistance(candidate, 5)

      print("Selected correct version:")
      for suggested_word in enumerate(suggested_words):
        print(suggested_word)
      selected_suggestion = int(input("Please give your response: "))
      MEMORY[candidate] = suggested_words[selected_suggestion]
      print('Your answer is saved as "{} -> {}"'.format(candidate, suggested_words[selected_suggestion]))
    ## YOUR CODE ENDS
    pass

In [54]:
AutoCorrect("word")

Your word is: gelmesm
Selected correct version:
(0, 'gelmesi')
Please give your response: 0
Your answer is saved as "gelmesm -> gelmesi"
